### 네이버 쇼핑 키워드 인사이트

#### 개인과제
- 키워드에 대한 검색량 데이터 수집
- 검색어에 대한 상품등록수 수집
- 수집한 데이터에 대한 인사이트 도출 : 예시) 경쟁률

#### 조별과제
- 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드와 상품 찾기
    - 예시외의 추가적인 페이지 데이터 수집하여 프로젝트 수행가능

#### 참고 서비스
- 신사임당 유투브 : https://youtu.be/Z0hL9khbpIw
- 응용 서비스 : https://whereispost.com/keyword, https://itemscout.io

#### 데이터 수집 프로세스
1. 네이버 검색광고 회원가입 및 로그인 : https://searchad.naver.com
2. 광고 시스템 > 도구 > API 사용관리
3. CUSTOMER_ID, API_KEY, SECRET_KEY 획득
4. 네이버 검색 광고 API 매뉴얼 사이트 참고 : http://naver.github.io/searchad-apidoc/#/guides
5. github 페이지 참고
    - https://github.com/naver/searchad-apidoc
    - https://github.com/naver/searchad-apidoc/blob/master/python-sample/examples/ad_management_sample.py
6. 키워드 입력하면 해당 키워드에 대한 데이터 수집

- hint
    - github 페이지의 코드에서 signaturehelper.py 파일을 이용하여 header를 만들어야 합니다.

In [1]:
import requests, json, time
import pandas as pd
import signaturehelper
from bs4 import BeautifulSoup
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

In [3]:
BASE_URL = 'https://api.naver.com'
API_KEY = '0100000000f98df504bbf667ca56999289b02a7c5d6a7f8faade8d8a0714e3055e716aee5d'
SECRET_KEY = 'AQAAAAD5jfUEu/ZnylaZkomwKnxdOPK62nRqM/7S+TGcDb6ntg=='
CUSTOMER_ID = '2488435'

#### 1. 개별과제 : 키워드에 대한 검색량 데이터 수집

In [4]:
def keyword_master(keyword):
    uri = '/keywordstool'
    method = 'GET'
    r = requests.get(BASE_URL + uri+f'?hintKeywords={keyword}&showDetail=1',
                     headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
    return  pd.DataFrame(r.json()['keywordList'])

In [5]:
keyword = "실내자전거"
df = keyword_master(keyword)

In [6]:
df.tail(2)

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
1008,종아리운동,450,3230,0.1,1.0,0.04,0.04,15,높음
1009,암슬리브,190,870,0.2,0.0,0.12,0.00,15,높음


In [7]:
df.head()

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,실내자전거,19400,163400,112.1,2643.0,0.66,1.76,15,높음
1,하체운동기구,1070,7790,8.0,120.8,0.81,1.66,15,높음
2,헬스자전거,1240,11900,17.4,318.0,1.56,2.86,15,높음
3,스피닝자전거,2330,24200,12.5,360.5,0.58,1.56,15,높음
4,스쿼트머신,2260,16800,5.0,195.0,0.24,1.23,15,높음


#### 2. 개별과제 : 검색어에 대한 상품등록수 수집
- item_count : 등록된 상품수
- total_search_count : PC, Mobile 을 합친 검색량

#### All

In [8]:
# 전체 등록 상품수를 int형으로 가져오는 코드
def all_item_cnt(keyword) :
    url = f'https://search.shopping.naver.com/search/all?query={keyword}'
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")
    result = dom.select('.subFilter_num__2x0jq')
    
    # 검색 결과가 없는 경우 0으로 return
    if result :
        return int(dom.select('.subFilter_num__2x0jq')[0].text.replace(',',''))
    else :
        return 0

In [9]:
# 검색 결과가 없는 상품 예시
keyword = '은성SP7300'
all_item_cnt(keyword)

0

#### 소호몰

In [10]:
# 소호 쇼핑몰에 등록되어 있는 상품수 

def soho_item_cnt(keyword) :
    url = f'https://search.shopping.naver.com/api/filter/mall?query={keyword}&mallFilterSection=SOHO&isQuerySearch=true&isDisplayCategory=false'
    response = requests.get(url)
    
    soho = response.json()['filterValues']
    
    cnt = 0
    for i in soho :
        cnt += i['productCount']
        
    return cnt

In [11]:
keyword = '은성SP7300'
soho_item_cnt(keyword)

0

In [12]:
# 1. kyeword로 입력한 검색량 데이터를 구한다
# 2. keyword를 통해 나온 모든 데이터의 상품수를 구한다.
# 시간이 많이 걸릴 수 있음으로 keywords 수를 정해서 상위 검색 키워드(100개)만 생성했습니다.
def item_count(keyword):
    df = keyword_master(keyword)[['relKeyword','monthlyPcQcCnt','monthlyMobileQcCnt']]
    df = df[:100]
    
    df['item_count'] = df['relKeyword'].apply(all_item_cnt)
    df['soho_item_count'] = df['relKeyword'].apply(soho_item_cnt)
    
    df.loc[df['monthlyMobileQcCnt']=='< 10', 'monthlyMobileQcCnt'] = 0
    df.loc[df['monthlyPcQcCnt']=='< 10', 'monthlyPcQcCnt'] = 0
    
    df['monthlyPcQcCnt'] = df['monthlyPcQcCnt'].astype(float)
    df['monthlyMobileQcCnt'] = df['monthlyMobileQcCnt'].astype(float)
    
    df['total_search_count'] = df['monthlyPcQcCnt'] + df['monthlyMobileQcCnt']
    return df

In [13]:
keyword = "실내자전거"
result_df = item_count(keyword)

In [14]:
result_df.head()

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,soho_item_count,total_search_count
0,실내자전거,19400.0,163400.0,255192,43934,182800.0
1,하체운동기구,1070.0,7790.0,57563,12183,8860.0
2,헬스자전거,1240.0,11900.0,248738,40425,13140.0
3,스피닝자전거,2330.0,24200.0,63127,17817,26530.0
4,스쿼트머신,2260.0,16800.0,46536,13628,19060.0


In [15]:
result_df.tail(2)

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,soho_item_count,total_search_count
98,자전거헬스,50.0,870.0,248727,40425,920.0
99,실내용자전거추천,10.0,40.0,14910,4939,50.0


#### 3. 개별과제 : 경쟁률
- 아이템수/검색수 : 높을수록 경쟁이 심한 상품 키워드
- 수집된 데이터에서 추가로 얻을수 있는 인사이트 추가
    - 논리적인 인사이트이면 인사이트 수준에 대한 평가는 하지 않음

In [16]:
result_df['competition_rate'] = result_df['item_count'] / result_df['total_search_count']
result_df['soho_competition_rate'] = result_df['soho_item_count'] / result_df['total_search_count']
result_df['total_competition_rate'] = result_df['competition_rate'] + result_df['soho_competition_rate']

In [17]:
# 스피닝자격증, 실내자전거대여의 키워드가 경쟁령이 낮다.
result_df.sort_values(by=['competition_rate'], axis=0).head(10)

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,soho_item_count,total_search_count,competition_rate,soho_competition_rate,total_competition_rate
41,스피닝자격증,10.0,90.0,1,1,100.0,0.010000,0.010000,0.020000
36,실내자전거대여,50.0,420.0,54,51,470.0,0.114894,0.108511,0.223404
49,스피닝협회,20.0,40.0,9,4,60.0,0.150000,0.066667,0.216667
26,스피닝강사,60.0,280.0,218,18,340.0,0.641176,0.052941,0.694118
11,실내자전거렌탈,90.0,750.0,597,551,840.0,0.710714,0.655952,1.366667
23,로잉머신렌탈,80.0,1820.0,1433,1331,1900.0,0.754211,0.700526,1.454737
22,앉아서하는운동,540.0,1570.0,2214,716,2110.0,1.049289,0.339336,1.388626
68,로잉머신,9160.0,53100.0,70462,20044,62260.0,1.131738,0.321940,1.453678
40,운동기구대여,50.0,410.0,547,388,460.0,1.189130,0.843478,2.032609
0,실내자전거,19400.0,163400.0,255192,43934,182800.0,1.396018,0.240339,1.636357


In [18]:
# 개인 스마트 스토어를 운영할 것이기 때문에 소호몰과 비교를 해봤다.
# 스피닝자격증와 스피닝강사 키워드가 경쟁력이 낮았습니다.
result_df.sort_values(by=['soho_competition_rate'], axis=0).head(10)

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,soho_item_count,total_search_count,competition_rate,soho_competition_rate,total_competition_rate
41,스피닝자격증,10.0,90.0,1,1,100.0,0.010000,0.010000,0.020000
26,스피닝강사,60.0,280.0,218,18,340.0,0.641176,0.052941,0.694118
49,스피닝협회,20.0,40.0,9,4,60.0,0.150000,0.066667,0.216667
36,실내자전거대여,50.0,420.0,54,51,470.0,0.114894,0.108511,0.223404
52,헬스장,22400.0,252600.0,389992,61121,275000.0,1.418153,0.222258,1.640411
0,실내자전거,19400.0,163400.0,255192,43934,182800.0,1.396018,0.240339,1.636357
17,폼롤러,14900.0,115300.0,332204,39588,130200.0,2.551490,0.304055,2.855545
68,로잉머신,9160.0,53100.0,70462,20044,62260.0,1.131738,0.321940,1.453678
21,밸런스보드,6100.0,34600.0,108101,13524,40700.0,2.656044,0.332285,2.988329
22,앉아서하는운동,540.0,1570.0,2214,716,2110.0,1.049289,0.339336,1.388626


In [19]:
# 전체와 소호 경쟁률을 더해서 가장 합리적인 키워드를 찾아봤습니다.
# 스피닝자격증, 스피닝협회, 실내자전거 대여의 키워드가 경쟁률이 낮았습니다.
result_df.sort_values(by=['total_competition_rate'], axis=0).head(10)

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,soho_item_count,total_search_count,competition_rate,soho_competition_rate,total_competition_rate
41,스피닝자격증,10.0,90.0,1,1,100.0,0.010000,0.010000,0.020000
49,스피닝협회,20.0,40.0,9,4,60.0,0.150000,0.066667,0.216667
36,실내자전거대여,50.0,420.0,54,51,470.0,0.114894,0.108511,0.223404
26,스피닝강사,60.0,280.0,218,18,340.0,0.641176,0.052941,0.694118
11,실내자전거렌탈,90.0,750.0,597,551,840.0,0.710714,0.655952,1.366667
22,앉아서하는운동,540.0,1570.0,2214,716,2110.0,1.049289,0.339336,1.388626
68,로잉머신,9160.0,53100.0,70462,20044,62260.0,1.131738,0.321940,1.453678
23,로잉머신렌탈,80.0,1820.0,1433,1331,1900.0,0.754211,0.700526,1.454737
0,실내자전거,19400.0,163400.0,255192,43934,182800.0,1.396018,0.240339,1.636357
52,헬스장,22400.0,252600.0,389992,61121,275000.0,1.418153,0.222258,1.640411


##### 결론
- 스피닝 자격증을 취득을 위한 상품이 경쟁률이 둘다 낮을 것으로 스피닝 강사면 자격증 취득을 위한 상품을 출시하는 것도 좋아보인다.
- 실제 실내 자전거를 유통하는 쇼핑몰을 만들고 싶다면 실내자전거 대여 서비스를 상품으로 진출하는 것도 좋아보인다.
- 그 외 스피닝협회, 스피닝강사와 같은 키워드로 소비자들을 모으는 것도 하나의 방법으로 보인다.

#### 4. 조별과제 : 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드 찾기
- 정답이 있는것은 아닙니다
- 여러가지 가정과 데이터를 수집, 분석하여 판매할 상품과 키워드를 선정해 보시면 됩니다.
- 앞서 2일간 했던 프로젝트와 같은 방법으로 수행하시면 됩니다.

### 평가기준

#### 개인평가(70)

    - 기본 문제 해결(30)
        - 데이터 수집 방법 : 10점
            - 최적의 수집 방법을 사용했는지 확인
        - 수집된 데이터의 결과 : 20점
            - 1번: 키워드에 대한 검색량 데이터 수집 : 15점
            - 2번: 검색어에 대한 상품등록수 수집 : 5점
            
    - 적절한 주석 및 설명(10)
        - 코드의 가독성 및 코드의 설명 : 10점
        
    - 코드 오류(10)
        - 발생 오류마다 : -2점
        
    - 추가 분석(20)
        - 인사이트 도출
            - 경쟁률 데이터 출력 : 10점
            - 추가 인사이트 : 10점
            
#### 조별평가(30)
    
    - 비지니스 인사이트 도출(5)
        - 스마트 스토어 운영자 입장에서의 추가 인사이트 도출
        - 예시) 선정한 상품으로 소호몰에서 1위 하였을경우 예상되는 매출
        - 인사이트 도출을 위한 프로세스
        
    - 실습문제의 분석(5)
        - 제출 자료가 잘 정리되고 판매 상품과 키워드를 논리적으로 잘 선정
        - 상품과 키워드 선정 자동화 > 가독성 좋은 키워드와 상품 선정 프로세스
        
    - 실습문제의 적절한 정리(20)
        - 프로젝트 주제(상품과 키워드 선정)에 맞는 논리적 인사이트가 있는지 확인
        - 프로젝트 수행 후 추후 과제 및 느낀점